In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from contractions import contractions_dict
from nltk.corpus import stopwords
from itertools import filterfalse
from nltk import pos_tag
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
data=pd.read_csv('emails.csv')

In [3]:
data

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [4]:
def strip_titles(text):
    if 'Subject: re :' in text:
        return text[13:]
    elif 'Subject: news :' in text:
        return text[15:]
    else:
        return text[8:]

In [5]:
data['text'] = data['text'].apply(lambda x:strip_titles(x))

In [6]:
data

,text,spam
0,naturally irresistible your corporate identit...,1
1,the stock trading gunslinger fanny is merril...,1
2,unbelievable new homes made easy im wanting ...,1
3,4 color printing special request additional ...,1
4,"do not have money , get software cds from her...",1
...,...,...
5723,research and development charges to gpg here...,0
5724,"receipts from visit jim , thanks again for ...",0
5725,enron case study update wow ! all on the sam...,0
5726,"interest david , please , call shirley cren...",0


In [7]:
data['text'] = data['text'].apply(lambda x:word_tokenize(x))

In [8]:
data

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[4, color, printing, special, request, additio...",1
4,"[do, not, have, money, ,, get, software, cds, ...",1
...,...,...
5723,"[research, and, development, charges, to, gpg,...",0
5724,"[receipts, from, visit, jim, ,, thanks, again,...",0
5725,"[enron, case, study, update, wow, !, all, on, ...",0
5726,"[interest, david, ,, please, ,, call, shirley,...",0


In [9]:
data['text'][0]

['naturally',
 'irresistible',
 'your',
 'corporate',
 'identity',
 'lt',
 'is',
 'really',
 'hard',
 'to',
 'recollect',
 'a',
 'company',
 ':',
 'the',
 'market',
 'is',
 'full',
 'of',
 'suqgestions',
 'and',
 'the',
 'information',
 'isoverwhelminq',
 ';',
 'but',
 'a',
 'good',
 'catchy',
 'logo',
 ',',
 'stylish',
 'statlonery',
 'and',
 'outstanding',
 'website',
 'will',
 'make',
 'the',
 'task',
 'much',
 'easier',
 '.',
 'we',
 'do',
 'not',
 'promise',
 'that',
 'havinq',
 'ordered',
 'a',
 'iogo',
 'your',
 'company',
 'will',
 'automaticaily',
 'become',
 'a',
 'world',
 'ieader',
 ':',
 'it',
 'isguite',
 'ciear',
 'that',
 'without',
 'good',
 'products',
 ',',
 'effective',
 'business',
 'organization',
 'and',
 'practicable',
 'aim',
 'it',
 'will',
 'be',
 'hotat',
 'nowadays',
 'market',
 ';',
 'but',
 'we',
 'do',
 'promise',
 'that',
 'your',
 'marketing',
 'efforts',
 'will',
 'become',
 'much',
 'more',
 'effective',
 '.',
 'here',
 'is',
 'the',
 'list',
 'of',


In [10]:
def normalize_token(list_of_token):
    return map(lambda x: x.lower(),list_of_token)

In [11]:
data['text']=data['text'].apply(lambda x: normalize_token(x))

In [12]:
data['text']=data['text'].apply(lambda x: list(x))

In [13]:
data

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[4, color, printing, special, request, additio...",1
4,"[do, not, have, money, ,, get, software, cds, ...",1
...,...,...
5723,"[research, and, development, charges, to, gpg,...",0
5724,"[receipts, from, visit, jim, ,, thanks, again,...",0
5725,"[enron, case, study, update, wow, !, all, on, ...",0
5726,"[interest, david, ,, please, ,, call, shirley,...",0


In [14]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [15]:
contracted_word_expansion("shouldn't")

'should not'

In [16]:
def contractions_expansion(list_of_token):
    return list(map(contracted_word_expansion,list_of_token))

In [17]:
data['text']=data['text'].apply(lambda x: contractions_expansion(x))

In [18]:
data

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[4, color, printing, special, request, additio...",1
4,"[do, not, have, money, ,, get, software, cds, ...",1
...,...,...
5723,"[research, and, development, charges, to, gpg,...",0
5724,"[receipts, from, visit, jim, ,, thanks, again,...",0
5725,"[enron, case, study, update, wow, !, all, on, ...",0
5726,"[interest, david, ,, please, ,, call, shirley,...",0


In [19]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [20]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [21]:
def filter_waste_words(list_of_token):
    return filterfalse(waste_word_or_not,list_of_token)

In [22]:
data['text']=data['text'].apply(lambda x: filter_waste_words(x))

In [23]:
data['text']=data['text'].apply(lambda x: list(x))

In [24]:
data

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[color, printing, special, request, additional...",1
4,"[do, not, have, money, get, software, cds, fro...",1
...,...,...
5723,"[research, and, development, charges, to, gpg,...",0
5724,"[receipts, from, visit, jim, thanks, again, fo...",0
5725,"[enron, case, study, update, wow, all, on, the...",0
5726,"[interest, david, please, call, shirley, crens...",0


In [25]:
data['text'][0]

['naturally',
 'irresistible',
 'your',
 'corporate',
 'identity',
 'lt',
 'is',
 'really',
 'hard',
 'to',
 'recollect',
 'a',
 'company',
 'the',
 'market',
 'is',
 'full',
 'of',
 'suqgestions',
 'and',
 'the',
 'information',
 'isoverwhelminq',
 'but',
 'a',
 'good',
 'catchy',
 'logo',
 'stylish',
 'statlonery',
 'and',
 'outstanding',
 'website',
 'will',
 'make',
 'the',
 'task',
 'much',
 'easier',
 'we',
 'do',
 'not',
 'promise',
 'that',
 'havinq',
 'ordered',
 'a',
 'iogo',
 'your',
 'company',
 'will',
 'automaticaily',
 'become',
 'a',
 'world',
 'ieader',
 'it',
 'isguite',
 'ciear',
 'that',
 'without',
 'good',
 'products',
 'effective',
 'business',
 'organization',
 'and',
 'practicable',
 'aim',
 'it',
 'will',
 'be',
 'hotat',
 'nowadays',
 'market',
 'but',
 'we',
 'do',
 'promise',
 'that',
 'your',
 'marketing',
 'efforts',
 'will',
 'become',
 'much',
 'more',
 'effective',
 'here',
 'is',
 'the',
 'list',
 'of',
 'clear',
 'benefits',
 'creativeness',
 'hand',

In [26]:
def split(list_of_token):
    return map(lambda x: re.split(regex,x)[0],list_of_token)

In [27]:
data['text'] = data['text'].apply(lambda x: split(x))

In [28]:
data['text'] = data['text'].apply(lambda x: list(x))

In [29]:
data

,text,spam
0,"[naturally, irresistible, your, corporate, ide...",1
1,"[the, stock, trading, gunslinger, fanny, is, m...",1
2,"[unbelievable, new, homes, made, easy, im, wan...",1
3,"[color, printing, special, request, additional...",1
4,"[do, not, have, money, get, software, cds, fro...",1
...,...,...
5723,"[research, and, development, charges, to, gpg,...",0
5724,"[receipts, from, visit, jim, thanks, again, fo...",0
5725,"[enron, case, study, update, wow, all, on, the...",0
5726,"[interest, david, please, call, shirley, crens...",0


In [30]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [31]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [32]:
def stopwords_removal(list_of_token):
    return filter(is_stopword,list_of_token)

In [33]:
data['text'] = data['text'].apply(lambda x: stopwords_removal(x))

In [34]:
data['text'] = data['text'].apply(lambda x: list(x))

In [35]:
data

,text,spam
0,"[naturally, irresistible, corporate, identity,...",1
1,"[stock, trading, gunslinger, fanny, merrill, m...",1
2,"[unbelievable, new, homes, easy, im, wanting, ...",1
3,"[color, printing, special, request, additional...",1
4,"[money, software, cds, software, compatibility...",1
...,...,...
5723,"[research, development, charges, gpg, forwarde...",0
5724,"[receipts, visit, jim, thanks, invitation, vis...",0
5725,"[enron, case, study, update, wow, day, super, ...",0
5726,"[interest, david, shirley, crenshaw, assistant...",0


In [36]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    else:
        (treebank_tag[0],wordnet.NOUN)

In [37]:
def get_pos_tag(list_of_token):
    return map(get_wnet_pos_tag,pos_tag(list_of_token))

In [38]:
data['text'] = data['text'].apply(lambda x: get_pos_tag(x))

In [39]:
data['text'] = data['text'].apply(lambda x: list(x))

In [40]:
data

,text,spam
0,"[(naturally, r), (irresistible, a), (corporate...",1
1,"[(stock, n), (trading, n), (gunslinger, n), (f...",1
2,"[(unbelievable, a), (new, a), (homes, n), (eas...",1
3,"[(color, n), (printing, v), (special, a), (req...",1
4,"[(money, n), (software, n), (cds, n), (softwar...",1
...,...,...
5723,"[(research, n), (development, n), (charges, n)...",0
5724,"[(receipts, n), (visit, v), (jim, a), (thanks,...",0
5725,"[(enron, n), (case, n), (study, n), (update, a...",0
5726,"[(interest, n), (david, n), (shirley, n), (cre...",0


In [41]:
lemmatizer=WordNetLemmatizer()

In [42]:
def token_lemmatization(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [43]:
def lemmatization(list_of_token):
    if len(list_of_token) > 0:
        return map(lambda x: token_lemmatization(x),list_of_token)

In [44]:
data['text'] = data['text'].apply(lambda x: lemmatization(x))

In [45]:
data['text'] = data['text'].apply(lambda x: list(x))

In [46]:
data

,text,spam
0,"[naturally, irresistible, corporate, identity,...",1
1,"[stock, trading, gunslinger, fanny, merrill, m...",1
2,"[unbelievable, new, home, easy, im, want, home...",1
3,"[color, print, special, request, additional, i...",1
4,"[money, software, cd, software, compatibility,...",1
...,...,...
5723,"[research, development, charge, gpg, forward, ...",0
5724,"[receipt, visit, jim, thanks, invitation, visi...",0
5725,"[enron, case, study, update, wow, day, super, ...",0
5726,"[interest, david, shirley, crenshaw, assistant...",0


In [47]:
vocab=set()
for list_of_token in data['text']:
    vocab=vocab.union(set(list_of_token))

In [48]:
len(vocab)

29195

In [49]:
vocab = list(vocab)

In [50]:
vocab.pop(0)

''

In [51]:
vocab_dict = dict(zip(vocab,list(range(0,len(vocab)))))

In [52]:
def join_tokens(list_of_token):
    return " ".join(list_of_token)

In [53]:
data['text'] = data['text'].apply(lambda x: join_tokens(x))

In [54]:
corpus = list()
for email_text in data['text']:
    corpus.append(email_text)

In [55]:
vectorizer = TfidfVectorizer(vocabulary=vocab_dict)
tf_idf_matrix = vectorizer.fit_transform(corpus)

In [56]:
tf_idf_matrix = tf_idf_matrix.toarray()

In [57]:
df = pd.DataFrame(tf_idf_matrix)
df['spam']=data['spam']

In [58]:
df

,0,1,2,3,4,5,6,7,8,9,...,29185,29186,29187,29188,29189,29190,29191,29192,29193,spam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
from Rashtriya_Raksha_University_Gaussian_NB import rru_gaussian_nb

In [ ]:
naive_bayes_configs = dict()
for n_components in np.arange(150,29194,2500):
    naive_bayes_configs[(n_components)] = rru_gaussian_nb(df,df['spam'],(0.7,0.0,0.3),True,n_components)

In [ ]:
naive_bayes_configs

In [59]:
df['spam'] = data['spam']

In [60]:
pca = PCA(n_components=50)

In [61]:
tf_idf_matrix_reduced = pca.fit_transform(tf_idf_matrix)

In [62]:
df = pd.DataFrame(data=tf_idf_matrix_reduced)

In [63]:
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.151221,0.178143,-0.234172,0.318166,0.006986,-0.006347,0.109024,0.062284,0.033068,-0.051089,...,-0.019994,0.047337,-0.061228,0.019185,-0.043573,-0.020896,-0.023568,-0.046182,-0.016919,0.006070
1,-0.077250,0.038654,0.010544,-0.038706,-0.022441,-0.014739,-0.008339,-0.018153,0.012398,0.036476,...,-0.013599,-0.020832,0.001186,-0.016510,-0.002983,0.011719,-0.011250,0.002615,-0.012886,-0.002494
2,-0.106523,0.065950,0.019640,0.000570,-0.012159,0.014742,0.053911,-0.017147,0.054453,-0.013744,...,-0.027672,0.011417,-0.093875,0.030063,0.019268,0.006187,-0.004464,0.011326,-0.068496,0.019377
3,-0.112584,0.050672,0.116491,0.059541,-0.054804,0.080401,-0.026788,-0.067900,-0.032808,-0.022527,...,0.031042,0.005231,-0.014279,-0.022604,-0.025776,0.010846,-0.004306,0.010901,0.023235,0.003477
4,-0.108247,0.084340,0.039410,-0.086155,-0.093342,-0.038114,-0.057890,0.100371,0.001002,0.015882,...,0.010271,0.029537,0.005851,-0.032914,0.023729,-0.006170,-0.001232,0.021097,-0.009919,-0.020663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,0.401539,0.074831,0.020238,0.019525,-0.029365,-0.015948,0.007496,0.004642,0.003875,0.000347,...,0.011970,0.002723,0.020936,0.000005,-0.045348,-0.043355,0.036452,-0.006749,0.036179,-0.027747
5724,-0.047198,-0.099706,-0.082764,-0.068445,-0.093340,0.098889,0.005407,-0.043596,0.026505,-0.021805,...,0.012949,-0.011982,-0.009592,-0.011831,-0.050752,0.053425,0.040051,0.027909,-0.026660,0.154202
5725,0.105273,-0.155596,-0.150923,-0.077352,-0.117501,0.189573,-0.039674,-0.065505,0.266495,-0.059946,...,-0.008335,0.006247,0.062035,0.011533,0.038561,-0.087186,-0.050196,-0.053734,0.050709,-0.016906
5726,0.360914,0.106249,-0.033263,-0.053314,-0.053486,0.005285,0.026920,-0.050886,-0.069015,-0.030114,...,-0.018272,-0.006490,-0.027420,-0.017404,0.045558,-0.004748,-0.003624,-0.004284,0.013961,-0.026746


In [64]:
df = pd.DataFrame(data=tf_idf_matrix_reduced)
df['spam']=data['spam']

In [65]:
from imblearn.over_sampling import SMOTE

In [66]:
smote_obj=SMOTE(sampling_strategy='minority',k_neighbors=5)

In [67]:
x_oversampled,y_oversampled=smote_obj.fit_resample(X=tf_idf_matrix_reduced,y=df['spam'])

In [68]:
df=pd.DataFrame(data=x_oversampled)
df['spam']=y_oversampled

In [69]:
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,spam
0,-0.151221,0.178143,-0.234172,0.318166,0.006986,-0.006347,0.109024,0.062284,0.033068,-0.051089,...,0.047337,-0.061228,0.019185,-0.043573,-0.020896,-0.023568,-0.046182,-0.016919,0.006070,1
1,-0.077250,0.038654,0.010544,-0.038706,-0.022441,-0.014739,-0.008339,-0.018153,0.012398,0.036476,...,-0.020832,0.001186,-0.016510,-0.002983,0.011719,-0.011250,0.002615,-0.012886,-0.002494,1
2,-0.106523,0.065950,0.019640,0.000570,-0.012159,0.014742,0.053911,-0.017147,0.054453,-0.013744,...,0.011417,-0.093875,0.030063,0.019268,0.006187,-0.004464,0.011326,-0.068496,0.019377,1
3,-0.112584,0.050672,0.116491,0.059541,-0.054804,0.080401,-0.026788,-0.067900,-0.032808,-0.022527,...,0.005231,-0.014279,-0.022604,-0.025776,0.010846,-0.004306,0.010901,0.023235,0.003477,1
4,-0.108247,0.084340,0.039410,-0.086155,-0.093342,-0.038114,-0.057890,0.100371,0.001002,0.015882,...,0.029537,0.005851,-0.032914,0.023729,-0.006170,-0.001232,0.021097,-0.009919,-0.020663,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,-0.097182,0.051782,0.012867,-0.050251,-0.027657,0.001184,-0.006742,-0.034927,0.015939,0.020653,...,-0.025624,-0.048479,0.033726,-0.021876,0.035514,-0.039825,-0.025494,-0.019357,-0.023388,1
8716,-0.100547,0.035491,0.001424,-0.022445,-0.016878,-0.020686,0.053819,0.003165,0.006939,0.058196,...,0.017069,-0.026457,0.004325,0.015430,-0.020158,-0.030746,-0.018794,0.004313,-0.003228,1
8717,-0.119935,0.046204,0.087026,0.025540,-0.054406,0.042193,0.042646,-0.057983,-0.076064,-0.013282,...,-0.060434,0.030590,-0.029066,-0.034200,-0.029137,-0.052575,-0.032065,-0.012117,-0.075975,1
8718,-0.122379,0.108140,0.075832,-0.140231,-0.175672,-0.078110,-0.041065,0.215339,-0.014392,0.024620,...,0.122776,0.034602,-0.030453,0.103611,-0.020158,0.057751,0.050571,0.004259,-0.046391,1


In [70]:
df[df['spam']==0].shape

(4360, 51)

In [71]:
df[df['spam']==1].shape

(4360, 51)

In [ ]:
training_data_len=int(0.7*df.shape[0])

In [ ]:
training_data_len

In [ ]:
zero_training_data=df[df['spam']==0].iloc[0:training_data_len//2]
one_training_data=df[df['spam']==1].iloc[0:training_data_len//2]

In [ ]:
zero_training_data.shape

In [ ]:
one_training_data.shape

In [ ]:
training_df=pd.concat([zero_training_data,one_training_data])

In [ ]:
training_df

In [ ]:
testing_data_len= int(0.3*df.shape[0])

In [ ]:
testing_data_len

In [73]:
X=df
y=df['spam']

In [74]:
X

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,spam
0,-0.151221,0.178143,-0.234172,0.318166,0.006986,-0.006347,0.109024,0.062284,0.033068,-0.051089,...,0.047337,-0.061228,0.019185,-0.043573,-0.020896,-0.023568,-0.046182,-0.016919,0.006070,1
1,-0.077250,0.038654,0.010544,-0.038706,-0.022441,-0.014739,-0.008339,-0.018153,0.012398,0.036476,...,-0.020832,0.001186,-0.016510,-0.002983,0.011719,-0.011250,0.002615,-0.012886,-0.002494,1
2,-0.106523,0.065950,0.019640,0.000570,-0.012159,0.014742,0.053911,-0.017147,0.054453,-0.013744,...,0.011417,-0.093875,0.030063,0.019268,0.006187,-0.004464,0.011326,-0.068496,0.019377,1
3,-0.112584,0.050672,0.116491,0.059541,-0.054804,0.080401,-0.026788,-0.067900,-0.032808,-0.022527,...,0.005231,-0.014279,-0.022604,-0.025776,0.010846,-0.004306,0.010901,0.023235,0.003477,1
4,-0.108247,0.084340,0.039410,-0.086155,-0.093342,-0.038114,-0.057890,0.100371,0.001002,0.015882,...,0.029537,0.005851,-0.032914,0.023729,-0.006170,-0.001232,0.021097,-0.009919,-0.020663,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8715,-0.097182,0.051782,0.012867,-0.050251,-0.027657,0.001184,-0.006742,-0.034927,0.015939,0.020653,...,-0.025624,-0.048479,0.033726,-0.021876,0.035514,-0.039825,-0.025494,-0.019357,-0.023388,1
8716,-0.100547,0.035491,0.001424,-0.022445,-0.016878,-0.020686,0.053819,0.003165,0.006939,0.058196,...,0.017069,-0.026457,0.004325,0.015430,-0.020158,-0.030746,-0.018794,0.004313,-0.003228,1
8717,-0.119935,0.046204,0.087026,0.025540,-0.054406,0.042193,0.042646,-0.057983,-0.076064,-0.013282,...,-0.060434,0.030590,-0.029066,-0.034200,-0.029137,-0.052575,-0.032065,-0.012117,-0.075975,1
8718,-0.122379,0.108140,0.075832,-0.140231,-0.175672,-0.078110,-0.041065,0.215339,-0.014392,0.024620,...,0.122776,0.034602,-0.030453,0.103611,-0.020158,0.057751,0.050571,0.004259,-0.046391,1


In [ ]:
zero_training_data=df[df['spam']==0].iloc[training_data_len//2:]
one_training_data=df[df['spam']==1].iloc[training_data_len//2:]

In [ ]:
zero_training_data.shape

In [ ]:
one_training_data.shape

In [ ]:
testing_df=pd.concat([zero_training_data,one_training_data])

In [ ]:
testing_df

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [76]:
gnb = GaussianNB()

In [77]:
#X_train = df.iloc[:,0:5000]
#y_train = df['spam']
X_train=df.iloc[:,0:2000]
y_train=df['spam']

In [78]:
X_test=df.iloc[:,0:2000]
y_test=df['spam']

In [ ]:
#gnb.fit(X=X_train,y=y_train)

In [ ]:
#predicted_categories = gnb.predict(X_train)

In [ ]:
#print(classification_report(y_true=y_train,y_pred=predicted_categories))

In [80]:
gnb.fit(X=X_train,y=y_train)

GaussianNB()

In [81]:
predicted_labels=gnb.predict(X=X_test)

In [82]:
predicted_labels

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [85]:
print('accuracy score:',accuracy_score(y_test,predicted_labels))

accuracy score: 1.0


In [83]:
print(classification_report(y_true=y_test,y_pred=predicted_labels))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4360
           1       1.00      1.00      1.00      4360

    accuracy                           1.00      8720
   macro avg       1.00      1.00      1.00      8720
weighted avg       1.00      1.00      1.00      8720



In [86]:
clf = LogisticRegression()

In [87]:
clf.fit(X=X_train,y=y_train)

LogisticRegression()

In [88]:
predicted_labels=gnb.predict(X=X_test)

In [89]:
predicted_labels

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [90]:
print('accuracy score:',accuracy_score(y_test,predicted_labels))

accuracy score: 1.0
